In [1]:
import numpy as np
import math
from scipy.stats import gamma
from sklearn.decomposition import IncrementalPCA

import tensorly as tl
from tensorly.cp_tensor import cp_mode_dot
import tensorly.tenalg as tnl
from tensorly.tenalg.core_tenalg import tensor_dot, batched_tensor_dot, outer, inner

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from pca import PCA

# Import TensorLy
import tensorly as tl
from tensorly.tenalg import kronecker
from tensorly import norm
from tensorly.decomposition import symmetric_parafac_power_iteration as sym_parafac
from tensorly.tenalg.core_tenalg.tensor_product import batched_tensor_dot
from tensorly.testing import assert_array_equal, assert_array_almost_equal

from tensorly.contrib.sparse.cp_tensor import cp_to_tensor

from tlda_final import TLDA
import cumulant_gradient
import tensor_lda_util as tl_util
## Break down into steps, then re-engineer.

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import stopwords

from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer
porter = PorterStemmer()

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        to_return = [porter.stem(self.wnl.lemmatize(t,get_wordnet_pos(t))) for t in word_tokenize(articles)]
        return to_return
    
class StemTokenizer(object):
    def __init__(self):
        self.porter = PorterStemmer()
    def __call__(self, articles):
        return [self.porter.stem(t) for t in word_tokenize(articles)]

from helpers import gtp 

        
import gc
from datetime import datetime
import random

import scipy

[nltk_data] Downloading package punkt to /home/adrien/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/adrien/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/adrien/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load tweets

import os
from helpers import load_tweets
filenames = ['../data/unzipped/' + name for name in os.listdir('../data/unzipped')]

all_tweets = load_tweets(filenames, preprocessor=None)

In [3]:
# use only 100000 tweets
tweets = random.sample(all_tweets, 100000)

In [4]:
# def stop words
n_samples = len(tweets)
print(n_samples)



100000


In [5]:
# countvec = CountVectorizer(tokenizer=gtp,
countvec = CountVectorizer(tokenizer=gtp,
                                strip_accents = 'unicode', # works
                                lowercase = True, # works
                                ngram_range = (1,2),
                                max_df = 0.4, # works
                                min_df = int(0.002*n_samples))

dtm = countvec.fit_transform(tweets)

In [6]:
dtm_sent = scipy.sparse.csr_matrix(dtm)


In [7]:
a = tl.tensor(dtm_sent.toarray(),dtype=np.float16)


In [8]:
del dtm_sent
gc.collect()

2

In [9]:
M1      = tl.mean(a, axis=0)


In [10]:
x_cent = scipy.sparse.csr_matrix(a - M1,dtype=np.float16) #center the data using the first moment 


In [11]:
gc.collect()


0

In [12]:

start = datetime.now()
print("now =", start)


batch_size = int(n_samples/20)
verbose = True
n_topic =  20

beta_0=0.003

pca = PCA(n_topic, beta_0, 30000)
pca.fit(x_cent) # fits PCA to  data, gives W
x_whit = pca.transform(x_cent) # produces a whitened words counts <W,x> for centered data x
now = datetime.now()
print("now =", now)
pca_time = now - start 

now = 2021-07-02 09:07:24.692810


KeyboardInterrupt: 

In [ ]:
gc.collect()
print(pca_time)

0:00:35.527440


In [ ]:
from importlib import reload  
import tlda_final
reload(tlda_final)
from tlda_final import TLDA


now = datetime.now()
print("now =", now)
learning_rate = 0.01 
batch_size =15000
t = TLDA(n_topic,n_senti=1, alpha_0= beta_0, n_iter_train=1000, n_iter_test=150, batch_size=batch_size,
         learning_rate=learning_rate)
now = datetime.now()
print("now =", now)

now = 2021-07-01 10:38:37.511187
now = 2021-07-01 10:38:37.512244


In [ ]:
now = datetime.now()
print("now =", now)

t.fit(x_whit,verbose=True) # fit whitened wordcounts to get decomposition of M3 through SGD

now = datetime.now()
print("now =", now)

now = 2021-07-01 10:38:37.543647
Epoch: 200
Epoch: 400
Epoch: 600
Epoch: 800
now = 2021-07-01 10:39:08.668651


In [ ]:
now = datetime.now()
print("now =", now)


t.factors_ = pca.reverse_transform(t.factors_)  # unwhiten the eigenvectors to get unscaled word-level factors

''' 
Recover alpha_hat from the eigenvalues of M3
'''  

eig_vals = [np.linalg.norm(k,3) for k in t.factors_ ]
# normalize beta
alpha      = np.power(eig_vals, -2)
print(alpha.shape)
alpha_norm = (alpha / alpha.sum()) * beta_0
t.alpha_   = alpha_norm
        
print(alpha_norm)

t.predict(x_whit,w_mat=True,doc_predict=False)  # normalize the factors 


now = datetime.now()
print("now =", now)

now = 2021-07-01 10:39:08.738405
(20,)
[2.19424417e-04 1.42362374e-04 1.51911362e-04 6.84582471e-05
 1.33031886e-04 2.55738134e-04 2.46100744e-04 2.11430843e-04
 1.73731484e-04 1.00908505e-04 1.45674253e-04 7.64351213e-05
 7.98640863e-05 2.47342123e-04 8.53057844e-05 1.35044509e-04
 2.07739737e-04 7.48047762e-05 1.11046955e-04 1.33644659e-04]
now = 2021-07-01 10:39:08.766175


In [ ]:
n_top_words=20
#print(t_n_indices)
n_sentiments = 1
for k in range(n_topic*n_sentiments):
    if k ==0:
        t_n_indices   =t.factors_[k,:].argsort()[:-n_top_words - 1:-1]
        top_words_JST = [i for i,v in countvec.vocabulary_.items() if v in t_n_indices]
    else:
        t_n_indices   =t.factors_[k,:].argsort()[:-n_top_words - 1:-1]
        top_words_JST = np.vstack([top_words_JST, [i for i,v in countvec.vocabulary_.items() if v in t_n_indices]])
        print([i for i,v in countvec.vocabulary_.items() if v in t_n_indices])

['look', 'like', 'donald', 'trump', 'mask', 'feel', 'shit', 'covid covid', 'covid coronavirus', 'presid trump', 'covid__', 'coronavirus covid', 'look like', 'act', 'administr', 'obama', 'quarantin', 'sound', 'coronavirus trump', 'feel like']
['trump', 'coronavirus', 'donald trump', 'lockdown', 'updat', 'extend', 'michigan', 'rise', 'coronavirus covid', 'lift', 'coronavirus lockdown', 'covid lockdown', 'india', 'case death', 'covid_', 'lockdown http', 'eas', 'coronavirus updat', 'coronavirus live', 'live updat']
['long', 'pandem', 'especi', 'help', 'save', 'corona', 'worker', 'work', 'servic', 'fight', 'world', 'happi', 'thank', 'hard', 'abl', 'famili', 'essenti', 'food', 'appreci', 'corona virus']
['time', 'infect', 'right', 'go', 'test', 'posit', 'actual', 'number', 'virus go', 'week', 'elonmusk', 'immun', 'antibodi', 'symptom', 'normal', 'negat', 'day', 'time http', 'mean', 'pass']
['time', 'right', 'talk', 'mind', 'actual', 'happen', 'global', 'year', 'vaccin', 'lose', 'read', 'arti

In [ ]:
now = datetime.now()



print("now =", now)
print(t.factors_.shape)
a_word       = tl.tensor(dtm.toarray(),dtype=tl.float32)

doc_topic_dist, topic_word_dist = t.predict(a_word,w_mat=False,doc_predict=True)
now = datetime.now()
 
print("now =", now)
end = datetime.now()
print(end)

now = 2021-07-01 10:12:49.977374
(20, 1078)
now = 2021-07-01 10:16:43.128848
2021-07-01 10:16:43.128974
